<a href="https://colab.research.google.com/github/lindsayhodge14/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [90.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [903 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports In

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-15 03:30:26--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.93MB/s    in 0.2s    

2022-08-15 03:30:27 (4.93 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REAKC26P07MDN|          5|            0|          0|   N|                Y|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|                Y|
|R14QJW3XF8QO1P|          5|            0|          0|   N|                Y|
|R2HB7AX0394ZGY|          5|            0|          0|   N|                Y|
| RGKMPDQGSAHR3|          5|            0|          0|   N|                Y|
|R1DJCVPQGCV66E|          5|            0|          0|   N|                Y|
|R3V52EAWLPBFQG|          3|            0|          0|   N|                Y|
|R3DKO8J1J28QBI|          2|            0|          0|   N|                Y|
| R764DBXGRNECG|          5|            1|          1|   N|                N|
| RW1853GAT0Z9F|          5|            0|          0|   N|     

*Deliverable 2*

In [27]:
# Step 1
from pyspark.sql.functions import col
total_votes_df = vine_df.filter(col("total_votes") >= 20)
total_votes_df.show()
print("Total number of reviews for Pet Products with >=20 reviews:", total_votes_df.count())

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|
|R3A1VQ3SQDXEJ3|          5|           20|         20|   N|                Y|
|  R39GSNW76GYF|          1|           20|         23|   N|                Y|
| RPJLR6MFDPXXE|          5|           35|         36|   N|     

In [9]:
#Step 2
majority_helpful_votes_df = total_votes_df.withColumn('percent_helpful_votes',col('helpful_votes')/col('total_votes')).alias('percent_helful_votes').filter(col("percent_helpful_votes") >= 0.5)
majority_helpful_votes_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percent_helpful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|   0.8709677419354839|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|                  1.0|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|   0.9354838709677419|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|   0.8333333333333334|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|   0.9642857142857143|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|              0.96875|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|   0.8372093023255814|


In [28]:
#Step 3
vine_participant_df = majority_helpful_votes_df.filter(col("vine") >= 'Y')
vine_participant_df.show()
print("Total number of vine participants for Pet Products with >=20 reviews:", vine_participant_df.count())

+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percent_helpful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|R3A71VR1JZD8WF|          2|           27|         30|   Y|                N|                  0.9|
|R16OMUJIGI18JZ|          5|           72|         72|   Y|                N|                  1.0|
|R3TS8ZP2FHQ9XR|          5|           39|         42|   Y|                N|   0.9285714285714286|
|R2MHP919VZN7DI|          5|           29|         30|   Y|                N|   0.9666666666666667|
| RD2BCTVS59A5L|          2|           20|         20|   Y|                N|                  1.0|
|R1JUJLXQ2CMWKF|          4|           25|         26|   Y|                N|   0.9615384615384616|
|R2T7YE0IFI6N9L|          3|           24|         25|   Y|                N|                 0.96|


In [31]:
vine_participant_df.groupBy('star_rating').count().show()

+-----------+-----+
|star_rating|count|
+-----------+-----+
|          1|    6|
|          3|   27|
|          5|   65|
|          4|   56|
|          2|   16|
+-----------+-----+



In [29]:
#Step 4
no_vine_df = majority_helpful_votes_df.filter(col("vine") >= 'N')
no_vine_df.show()
print("Total number of people who provided unpaid reviews for Pet Products with >=20 reviews:", no_vine_df.count())

+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percent_helpful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|   0.8709677419354839|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|                  1.0|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|   0.9354838709677419|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|   0.8333333333333334|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|   0.9642857142857143|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|              0.96875|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|   0.8372093023255814|


In [32]:
no_vine_df.groupBy('star_rating').count().show()

+-----------+-----+
|star_rating|count|
+-----------+-----+
|          1| 7575|
|          3| 2738|
|          5|20677|
|          4| 4956|
|          2| 2064|
+-----------+-----+



In [51]:
#Step 5: The counts were completed throughout the previous steps
total_reviews=total_votes_df.count()
total_fivestar_reviews= total_votes_df.where(col("star_rating")==5).count()
total_vine_reviews= vine_participant_df.count()
total_no_vine_reviews= no_vine_df.count()
total_fivestar_paid= vine_participant_df.where(col("star_rating")==5).count()
total_fivestar_unpaid= no_vine_df.where(col("star_rating")==5).count()

percent_5star_paid=(vine_participant_df.where(col("star_rating")==5).count()/vine_participant_df.count()*100)
percent_5star_unpaid=(no_vine_df.where(col("star_rating")==5).count()/no_vine_df.count()*100)


In [52]:
total_reviews

39376

In [56]:
total_vine_reviews

170

In [57]:
total_no_vine_reviews

38010

In [53]:
total_fivestar_reviews

20797

In [58]:
total_fivestar_paid

65

In [59]:
total_fivestar_unpaid

20677

In [54]:
percent_5star_paid

38.23529411764706

In [55]:
percent_5star_unpaid

54.39884240989213